## Hola, en este notebook desarrollaremos el proyecto del curso "Introduction to Deep Learning & Neural Networks with Keras" 

In [1]:
#importamos libreria para analisis y manipulacion de datos
import pandas as pd
import numpy as np

#importamos la libreria para particion del conjunto de datos 
from sklearn.model_selection import train_test_split

#importamos librerias para el modelado
import keras
from keras.models import Sequential
from keras.layers import Dense
#importamos libreria para el analisis de las predicciones 
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [2]:
#descargamos el conjunto de datos que vamos a usar y revisamos los primeros 10 registros
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head(10)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
5,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.03
6,380.0,95.0,0.0,228.0,0.0,932.0,594.0,365,43.70
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.45
8,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
9,475.0,0.0,0.0,228.0,0.0,932.0,594.0,28,39.29


### Analizamos los datos para saber si existen tipos que no concuerden, valores perdidos, entre otros

Primero revisamos si existen valores perdidos

In [3]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Como podemos observar no existen valores faltantes asi que proseguimos a revisar el tipo de dato de las columnas

In [4]:
concrete_data.dtypes

Cement                float64
Blast Furnace Slag    float64
Fly Ash               float64
Water                 float64
Superplasticizer      float64
Coarse Aggregate      float64
Fine Aggregate        float64
Age                     int64
Strength              float64
dtype: object

No se encuentran incongruencias en los tipos de los datos asi proseguimos a hacer el resumen de las variables. ya que no sabemos las unidades que se usan en las variables no sabemos si debemos estandarizar los datos en las variables.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


De esta tabla podemos ver que existen 1030 registros 

### Ahora separamos la variable objetivo del resto

In [6]:
concrete_data_columns = concrete_data.columns

X = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # todas las variables menos la 'Strength'
y = concrete_data['Strength'] #la variable 'Strength'


In [7]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### Empezamos a modelar y crear una Neural Network

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.30)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [9]:
n_cols = X_train.shape[1]

## _Propiedades de la red A_

- hidden layer: One
- nodes: 10
- activation function: ReLU
- optimizer:  adam 
- loss function: mean squared error
- epochs: 50

In [10]:
# define regression model A
def regression_model_a():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Entrenamiento y Evaluacion del modelo

### Iteramos los pasos del 1 al 3 50 veces para poder determinar la media y desviacion estandar del mean_squared_error

In [11]:
mse_a =[]

for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.30)
        
    model_a = regression_model_a()
    model_a.fit(X_train, y_train, epochs = 50, verbose=0)
    y_pred = model_a.predict(X_test)
    
    mse_a.append(mean_squared_error(y_test, y_pred))
    

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:217

In [12]:
print('Mean: {:.2f}'.format(np.mean(mse_a)))
print('Standar Deviation: {:.2f}'.format(np.std(mse_a)))

Mean: 747.18
Standar Deviation: 754.58


## _Propiedades de la red B_

- hidden layer: One
- nodes: 10
- activation function: ReLU
- optimizer:  adam 
- loss function: mean squared error
- epochs: 50
- Normalized data

In [13]:
#Normalizamos las variables de entrenamiento y testeo

X_norm = (X - X.mean()) / X.std()
y_norm = (y - y.mean()) / y.std()

In [14]:
# define regression model B
def regression_model_b():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [15]:
mse_b =[]

for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, train_size = 0.30)
        
    model_b = regression_model_b()
    model_b.fit(X_train, y_train, epochs = 50, verbose=0)
    y_pred = model_b.predict(X_test)
    
    mse_b.append(mean_squared_error(y_test, y_pred))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:217

In [16]:
print('Mean: {:.2f}'.format(np.mean(mse_b)))
print('Standar Deviation: {:.2f}'.format(np.std(mse_b)))

Mean: 0.38
Standar Deviation: 0.08


## _Propiedades de la red C_

- hidden layer: One
- nodes: 10
- activation function: ReLU
- optimizer:  adam 
- loss function: mean squared error
- epochs: 100
- Normalized data

In [17]:
# define regression model c
def regression_model_c():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [18]:
mse_c =[]

for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, train_size = 0.30)
        
    model_c = regression_model_c()
    model_c.fit(X_train, y_train, epochs = 100, verbose=0)
    y_pred = model_c.predict(X_test)
    
    mse_c.append(mean_squared_error(y_test, y_pred))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:217

In [19]:
print('Mean: {:.2f}'.format(np.mean(mse_c)))
print('Standar Deviation: {:.2f}'.format(np.std(mse_c)))

Mean: 0.29
Standar Deviation: 0.06


## _Propiedades de la red D_

- hidden layer: three
- nodes: 10 each hidden layer
- activation function: ReLU
- optimizer:  adam 
- loss function: mean squared error
- epochs: 50
- Normalized data

In [20]:
# define regression model D
def regression_model_d():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [21]:
mse_d =[]

for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, train_size = 0.30)
        
    model_d = regression_model_d()
    model_d.fit(X_train, y_train, epochs = 50, verbose=0)
    y_pred = model_d.predict(X_test)
    
    mse_d.append(mean_squared_error(y_test, y_pred))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_split.py:217

In [22]:
print('Mean: {:.2f}'.format(np.mean(mse_d)))
print('Standar Deviation: {:.2f}'.format(np.std(mse_d)))

Mean: 0.32
Standar Deviation: 0.05
